# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
import json
from pymongo import MongoClient

In [2]:
# Create an instance of MongoClient
# MongoDB connection URI
mongo_uri = "mongodb://localhost:27017/"

# Create an instance of MongoClient
client = MongoClient(mongo_uri)


In [3]:
# confirm that our new database was created
# List available databases
database_names = client.list_database_names()

# Check if the desired database is in the list
if "uk_food" in database_names:
    print("The 'uk_food' database created.")
else:
    print("The 'uk_food' database not found.")


The 'uk_food' database created.


In [4]:
# assign the uk_food database to a variable name
db = client['uk_food']
collection = db['establishments']

In [5]:
# review the collections in our new database
collection_names = db.list_collection_names()

# Print the collection names
for collection_name in collection_names:
    print(collection_name)


establishments


In [6]:
# review a document in the establishments collection
document = collection.find_one()

# Print the retrieved document
print(document)


{'_id': ObjectId('64f06d5cbbe6397f1aafa728'), 'name': 'Penang Flavours', 'type': 'Halal', 'borough': 'Greenwich', 'address': {'building': '123', 'street': 'Greenwich Street', 'postcode': 'SE10 9XX'}, 'grades': [], 'cuisine': 'Malaysian'}


In [7]:
# assign the collection to a variable
# Assign the establishments collection to a variable
collection = db['establishments']


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "name": "Penang Flavours",
    "type": "Halal",
    "borough": "Greenwich",
    "address": {
        "building": "123",
        "street": "Greenwich Street",
        "postcode": "SE10 9XX"
    },
    "grades": [],
    "cuisine": "Malaysian"
}


In [9]:
# Insert the new restaurant into the collection
result = collection.insert_one(new_restaurant)

# Print the inserted document's ID
print("Inserted document ID:", result.inserted_id)


Inserted document ID: 64f5911b3b59c6aa943c7714


In [10]:
# Check that the new restaurant was inserted
query = {"name": "Penang Flavours"}
found_restaurant = collection.find_one(query)

if found_restaurant:
    print("New restaurant found:")
    print(found_restaurant)
else:
    print("New restaurant not found.")


New restaurant found:
{'_id': ObjectId('64f06d5cbbe6397f1aafa728'), 'name': 'Penang Flavours', 'type': 'Halal', 'borough': 'Greenwich', 'address': {'building': '123', 'street': 'Greenwich Street', 'postcode': 'SE10 9XX'}, 'grades': [], 'cuisine': 'Malaysian'}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

result = collection.find(query, projection)

# Print the results
for document in result:
    print(document)


3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
query = {"name": "Penang Flavours"}
update = {"$set": {"BusinessTypeID": "64f06d5cbbe6397f1aafa728"}}

# Perform the update
result = collection.update_one(query, update)

# Confirm that the new restaurant was updated
if result.modified_count > 0:
    print("Restaurant updated.")
else:
    print("Restaurant not found or failed.")


Restaurant updated.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}
count_dover_documents = collection.count_documents(query)

# Print the count
print("Number of 'Dover':", count_dover_documents)


Number of 'Dover': 0


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

# Delete documents with the specified query
result = collection.delete_many(query)

In [16]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Find documents with the specified query
remaining_dover_documents = collection.find(query)

# Check if any remaining documents were found
if remaining_dover_documents.count() > 0:
    print("There are still some 'Dover'.")
else:
    print("No more 'Dover' found.")


No more 'Dover' found.


C:\Users\Wolfred\AppData\Local\Temp\ipykernel_21512\389050994.py:8: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if remaining_dover_documents.count() > 0:


In [17]:
# Check that other documents remain with 'find_one'
remaining_document = collection.find_one()

# Check if a document was found
if remaining_document:
    print("There are still some remaining.")
    print(remaining_document)
else:
    print("No more lft.")


There are still some remaining.
{'_id': ObjectId('64f06d5cbbe6397f1aafa728'), 'name': 'Penang Flavours', 'type': 'Halal', 'borough': 'Greenwich', 'address': {'building': '123', 'street': 'Greenwich Street', 'postcode': 'SE10 9XX'}, 'grades': [], 'cuisine': 'Malaysian', 'BusinessTypeID': '64f06d5cbbe6397f1aafa728'}


5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude
from decimal import Decimal

# Query to find documents with longitude stored as string
query = {"longitude": {"$type": "string"}}

# Update operation to convert longitude to decimal
update = {"$set": {"longitude": Decimal("$longitude")}}

result = collection.update_many(query, update)


InvalidOperation: [<class 'decimal.ConversionSyntax'>]

In [21]:
# Query to find documents with latitude stored as string
query = {"latitude": {"$type": "string"}}

# Update operation to convert latitude to decimal
update = [
    {"$set": {"latitude": {"$toDouble": "$latitude"}}},
    {"$set": {"latitude": {"$convert": {"input": "$latitude", "to": "decimal"}}}}
]

result = collection.update_many(query, update)


In [22]:
# Check that the coordinates are now numbers
sample_documents = collection.find().limit(5)

# Print the coordinates of the sample documents
for document in sample_documents:
    longitude = document.get("longitude")
    latitude = document.get("latitude")
    print(f"Longitude: {longitude}, Latitude: {latitude}")


Longitude: None, Latitude: None
Longitude: None, Latitude: None
